### 6개키워드 중 하나만 나오게 하는 샘플모델링

## ./crawled_img/(분위기키워드 폴더명)
 - joyful(활발한)
 - adventure(모험적인)
 - cultural(문화적인)
 - tradition(전통적인)
 - nature(자연)
 - art(예술적인)

### CNN 사용한 샘플모델링(데이터 증강X)

In [1]:
import os
import cv2
import random
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# 이미지 크기 설정
img_size = (224, 224) # 이미지 크키가 크면 메모리를 많이 차지해서 설정 필요

# 데이터 로드 및 전처리 함수
def load_and_preprocess_data(image_paths, label, brightness=True):
    data = []
    labels = []

    # 랜덤하게 100장만 선택
    selected_images = random.sample(image_paths, 100) #300개 이미지

    for img_path in selected_images:
        img = load_img(img_path, target_size=img_size)
        img_array = img_to_array(img)

        # 이미지 명도와 채도 높이기
        if brightness:
            img_array = cv2.convertScaleAbs(img_array, alpha=1.2, beta=30)

        img_array = img_array.astype('float32') / 255.0  # 이미지를 0과 1 사이 값으로 정규화

        data.append(img_array)
        labels.append(label)

    return np.array(data), np.array(labels)


# 카테고리 리스트
categories = ["joyful", "adventure", "tradition", "nature", "cultural", "art"]
num_classes = len(categories)

# 이미지 데이터 로드 및 전처리
all_data = []
all_labels = []

for i, category in enumerate(categories):
    category_path = os.path.join("crawled_img", category) # crawled_img/안에 있는 카테고리 리스트
    category_images = [os.path.join(category_path, img) for img in os.listdir(category_path)]
    
    data, labels = load_and_preprocess_data(category_images, label=i)
    all_data.extend(data)
    all_labels.extend(labels)

# 데이터 합치기 및 레이블 변환
X = np.array(all_data)
y = to_categorical(np.array(all_labels), num_classes=num_classes)

# 학습 및 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# CNN 모델 생성
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
15/15 [==============================] - 53s 4s/step - loss: 2.7246 - accuracy: 0.2167 - val_loss: 1.6795 - val_accuracy: 0.3417
Epoch 2/20
15/15 [==============================] - 48s 3s/step - loss: 1.5757 - accuracy: 0.3333 - val_loss: 1.5144 - val_accuracy: 0.3250
Epoch 3/20
15/15 [==============================] - 45s 3s/step - loss: 1.4213 - accuracy: 0.4313 - val_loss: 1.5768 - val_accuracy: 0.3750
Epoch 4/20
15/15 [==============================] - 46s 3s/step - loss: 1.3605 - accuracy: 0.4979 - val_loss: 1.4591 - val_accuracy: 0.4917
Epoch 5/20
15/15 [==============================] - 43s 3s/step - loss: 1.1661 - accuracy: 0.5646 - val_loss: 1.4295 - val_accuracy: 0.4250
Epoch 6/20
15/15 [==============================] - 43s 3s/step - loss: 1.0173 - accuracy: 0.6271 - val_loss: 1.3254 - val_accuracy: 0.5000
Epoch 7/20
15/15 [==============================] - 41s 3s/step - loss: 0.8664 - accuracy: 0.6833 - val_loss: 1.4641 - val_accuracy: 0.4917
Epoch 8/20
15/15 [==

### CNN 사용한 샘플모델링(데이터 증강) - 이미지 100장

In [33]:
import os
import cv2
import random
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# 이미지 크기 설정
img_size = (224, 224)

# 데이터 로드 및 전처리 함수
def load_and_preprocess_data(image_paths, label, brightness=True):
    data = []
    labels = []

    # 랜덤하게 100장만 선택
    selected_images = random.sample(image_paths, 100)

    for img_path in selected_images:
        img = cv2.imread(img_path)
        
        # 이미지가 비어 있는지 확인
        if img is None:
            print(f"이미지 로딩 중 오류 발생: {img_path}")
            continue

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, img_size)

        # 이미지 명도와 채도 높이기
        if brightness:
            img = cv2.convertScaleAbs(img, alpha=1.2, beta=30)

        img_array = img.astype('float32') / 255.0  # 이미지를 0과 1 사이 값으로 정규화

        data.append(img_array)
        labels.append(label)

    return np.array(data), np.array(labels)

# 카테고리 리스트
categories = ["joyful", "adventure", "tradition", "nature", "cultural", "art"]
num_classes = len(categories)

# 데이터 로드 및 전처리 함수를 이용하여 데이터 읽기
all_data = []
all_labels = []

for i, category in enumerate(categories):
    category_path = os.path.join("crawled_img", category)
    category_images = [os.path.join(category_path, img) for img in os.listdir(category_path)]
    
    data, labels = load_and_preprocess_data(category_images, label=i)
    all_data.extend(data)
    all_labels.extend(labels)

# 데이터 합치기 및 레이블 변환
X = np.array(all_data)
y = to_categorical(np.array(all_labels), num_classes=num_classes)

# 학습 및 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# CNN 모델 생성
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 데이터 증강을 적용하여 모델 훈련
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train)
model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=20, validation_data=(X_test, y_test))


Epoch 1/20
15/15 [==============================] - 84s 5s/step - loss: 2.4894 - accuracy: 0.1583 - val_loss: 1.7986 - val_accuracy: 0.1417
Epoch 2/20
15/15 [==============================] - 71s 4s/step - loss: 1.7684 - accuracy: 0.1813 - val_loss: 1.6924 - val_accuracy: 0.2750
Epoch 3/20
15/15 [==============================] - 59s 4s/step - loss: 1.6336 - accuracy: 0.2854 - val_loss: 1.5882 - val_accuracy: 0.3917
Epoch 4/20
15/15 [==============================] - 56s 4s/step - loss: 1.5683 - accuracy: 0.3688 - val_loss: 1.8033 - val_accuracy: 0.3417
Epoch 5/20
15/15 [==============================] - 57s 4s/step - loss: 1.5395 - accuracy: 0.3417 - val_loss: 1.4774 - val_accuracy: 0.4000
Epoch 6/20
15/15 [==============================] - 54s 4s/step - loss: 1.5360 - accuracy: 0.3938 - val_loss: 1.4210 - val_accuracy: 0.3833
Epoch 7/20
15/15 [==============================] - 51s 3s/step - loss: 1.5132 - accuracy: 0.3646 - val_loss: 1.5818 - val_accuracy: 0.3833
Epoch 8/20
15/15 [==

### CNN 사용한 샘플모델링(데이터 증강) - 이미지 300장

In [34]:
import os
import cv2
import random
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# 이미지 크기 설정
img_size = (224, 224)

# 데이터 로드 및 전처리 함수
def load_and_preprocess_data(image_paths, label, brightness=True):
    data = []
    labels = []

    # 랜덤하게 300장만 선택
    selected_images = random.sample(image_paths, 300)

    for img_path in selected_images:
        img = cv2.imread(img_path)
        
        # 이미지가 비어 있는지 확인
        if img is None:
            print(f"이미지 로딩 중 오류 발생: {img_path}")
            continue

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, img_size)

        # 이미지 명도와 채도 높이기
        if brightness:
            img = cv2.convertScaleAbs(img, alpha=1.2, beta=30)

        img_array = img.astype('float32') / 255.0  # 이미지를 0과 1 사이 값으로 정규화

        data.append(img_array)
        labels.append(label)

    return np.array(data), np.array(labels)

# 카테고리 리스트
categories = ["joyful", "adventure", "tradition", "nature", "cultural", "art"]
num_classes = len(categories)

# 데이터 로드 및 전처리 함수를 이용하여 데이터 읽기
all_data = []
all_labels = []

for i, category in enumerate(categories):
    category_path = os.path.join("crawled_img", category)
    category_images = [os.path.join(category_path, img) for img in os.listdir(category_path)]
    
    data, labels = load_and_preprocess_data(category_images, label=i)
    all_data.extend(data)
    all_labels.extend(labels)

# 데이터 합치기 및 레이블 변환
X = np.array(all_data)
y = to_categorical(np.array(all_labels), num_classes=num_classes)

# 학습 및 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# CNN 모델 생성
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 데이터 증강을 적용하여 모델 훈련
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train)
model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=20, validation_data=(X_test, y_test))


Epoch 1/20
45/45 [==============================] - 172s 4s/step - loss: 2.0617 - accuracy: 0.1813 - val_loss: 1.7111 - val_accuracy: 0.2583
Epoch 2/20
45/45 [==============================] - 163s 4s/step - loss: 1.7504 - accuracy: 0.2410 - val_loss: 1.7167 - val_accuracy: 0.2833
Epoch 3/20
45/45 [==============================] - 157s 3s/step - loss: 1.6036 - accuracy: 0.3326 - val_loss: 1.4596 - val_accuracy: 0.4167
Epoch 4/20
45/45 [==============================] - 204s 5s/step - loss: 1.4978 - accuracy: 0.3840 - val_loss: 1.3543 - val_accuracy: 0.4556
Epoch 5/20
45/45 [==============================] - 153s 3s/step - loss: 1.3808 - accuracy: 0.4569 - val_loss: 1.5203 - val_accuracy: 0.3889
Epoch 6/20
45/45 [==============================] - 140s 3s/step - loss: 1.3289 - accuracy: 0.4646 - val_loss: 1.2940 - val_accuracy: 0.4667
Epoch 7/20
45/45 [==============================] - 144s 3s/step - loss: 1.2468 - accuracy: 0.5000 - val_loss: 1.6545 - val_accuracy: 0.3556
Epoch 8/20
45